# Rede Neural de Base Radial (RBF)

As redes RBF são redes de alimentação direta (feedforward) consistindo de três camadas:


1.   **Camada de entrada**: propaga os estímulos
2.   **Camada escondida**: Unidades de processamento localmente sintonizáveis, utilizando mapeamento não linear.
3.   **Camada de saída**: Unidades de processamento lineares.


****

**O treinamento dessa rede ocorre de forma híbrida**, combinando aprendizagem não supervisionada (ANS) com a supervisionada(AS). Isso ocorre, pois em geral não se sabe quais saídas se desejam para a camada escondida. Sendo assim, a distribuição de trabalhos ocorre:
*   **ANS**: Treina a camada escondida definindo seus parâmetros livres (centros, larguras dos campos receptivos e pesos).
*   **AS**: Determina os valores dos pesos entre as camadas escondidas e de saída, considerando constantes os parâmetros já definidos.


****

**O aprendizado consiste em** determinar os valores para:
*   centro das funções de base radial,
*   largura das funções,
*   pesos da camada de saída.


Além disso, para cada neurônio da camada escondida, ele computa uma função de base radial.


Os passos necessários são:
1.   Utilizar um algoritmo ANS para encontrar os centros (protótipo para um cluster) das RBF;
2.   Utilizar métodos heurísticos para determinar a largura (área de influência de um cluster) de cada função;
3.   Utilizar um AS para determinar os pesos da camada de saída da rede.

1ª Etapa: Inicialização dos grupos com K-Means

In [ ]:
!git clone https://github.com/andssuu/redes_neurais_pos.git

Cloning into 'redes_neurais_pos'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 59 (delta 10), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), done.


Definição da função de base radial

In [125]:
#função de base radial gaussiana
def rbfGaussiana(x, c, s):
    return np.exp(-1 / (2 * s**2) * (x-c)**2)

#função de cálculo da largura do campo receptivo em que se repete o mesmo valor pra todos os neurônios
def computeEqualStds(centers, k):
  dist = [np.sqrt(np.sum((c1 - c2) ** 2)) for c1 in centers for c2 in centers]
  dMax = np.max(dist)
  stds = np.repeat(dMax / np.sqrt(2 * k), k)
  return stds

2ª Etapa - Treinamento de uma Rede Neural

In [114]:
from sklearn.cluster import KMeans
import numpy as np

class RBFNet(object):
    """Implementation of a Radial Basis Function Network"""

    def __init__(self, k=3, attnumber=4, lr=0.01, epochs=100, rbf=rbfGaussiana, computeStds=computeEqualStds):
        self.k = k  # grupos ou numero de neuronios na camada escondida
        self.lr = lr # taxa de aprendizagem
        self.epochs = epochs  # número de iterações
        self.rbf = rbf # função de base radial
        self.computeStds = computeStds  #função de cálculo da largura do campo receptivo
        self.w = np.random.randn(self.k, attnumber)
        self.b = np.random.randn(1)

    def fit(self, X, y):
        self.stds = []
        #K-Means pra pegar os centros inicias 
        #1º parâmetro da rede RBF
        kmeans = KMeans(
            n_clusters=self.k, init='random',
            n_init=10, max_iter=300).fit(X)
        self.centers = kmeans.cluster_centers_
        #print('centers: ', self.centers)

        #Cálculo da largura do campo receptivo
        #2º parâmetro da rede RBF
        self.stds = self.computeStds(self.centers, self.k)
        # training
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                # forward pass
                #calcula a saída de cada neurônio da função de base radial
                phi = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                #calcula somatório do produto da saída da função de base radial e os pesos
                F = phi.T.dot(self.w)
                F = np.sum(F) + self.b
                #saída da rede
                out = 0 if F < 0 else 1
                #função de perda 
                loss = (y[i] - out).flatten() ** 2
                #print('Loss: {0:.2f}'.format(loss[0]))
                #cálculo do erro
                error = (y[i] - out).flatten()
                #atualização dos pesos
                #3º Parâmetro da rede 
                self.w = self.w + self.lr * error * phi 
                self.b = self.b + self.lr * error

    #calcula saída da rede RBF com a rede treinada
    def predict(self, X):
        y_pred = []
        error = 0
        for i in range(X.shape[0]):
            a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = a.T.dot(self.w)
            F = np.sum(F) + self.b
            out = 0 if F < 0 else 1
            y_pred.append(out)
        return np.array(y_pred)


# Executando a rede neural RBF

In [115]:
# Neste código vou utilizar o pandas, framework amplamente utilizado pra lidar com dados
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#carrega a base de dados e retorna conjuntos de treinamento e teste
def load_data():
    url = 'redes_neurais_pos/RBF/iris.csv'
    df = pd.read_csv(url)
    #remove a ultima coluna (dados)
    data = df[df.columns[:-1]]
    #normaliza os dados
    normalized_data = (data - data.min()) / (data.max() - data.min())
    #retorna a última coluna (rótulos)
    labels = df[df.columns[-1]]
    #separa em conjunto de treinamento e teste com seus respectivos rótulos
    X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test

#chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()

#transforma rótulos do conjunto de treinamento em numeros pra calculo do erro
le = preprocessing.LabelEncoder()
le.fit(training_labels.values)
training_labels_transformed = le.transform(training_labels.values)

#chama RBF
rbfnet = RBFNet(lr=1e-2, attnumber=4, k=3, computeStds=computeEqualStds)
rbfnet.fit(training_inputs.values, training_labels_transformed)

#transforma rótulos do conjunto de teste em numeros pra calculo do erro
le = preprocessing.LabelEncoder()
le.fit(test_labels.values)
test_labels_transformed = le.transform(test_labels.values)

#y_pred = rbfnet.predict(test_labels_transformed)
y_pred = rbfnet.predict(test_inputs.values)

errorabs = abs(test_labels_transformed-y_pred)

print('error: ', np.sum(errorabs)/len(test_labels_transformed))

error:  0.6666666666666666


# Descrição Mini Projeto

Utilizando o código acima, modifique a última seção (Executando com Base de Dados) para que ele seja executado com a base de dados do arquivo iris.csv. Depois, modifique a função de base radial implementada (Gaussiana) para a Multiquadrática Inversa e calcule a taxa de erro.


1- Calcular a quantidade de neurônios escondidos:

a) 3

b) 5

c) 7

d) 9

Qual foi a melhor configuração? Avaliaria um outro valor?

2- Utilizando a melhor configuração da questão anterior, calcular a taxa de erro usando uma das outras maneiras de retorno da largura do campo receptivo da função de base radial, em que cada neurônio possui sua própria largura. 


3- Calcular a taxa de erro combinando 2 funções de Base Radial e as duas maneiras de cálculo da largura do campo receptivo:

a) Gaussiana

b) Multiquadrática Inversa


Qual foi a melhor configuração?



DATA DE ENTREGA: 25/11/2020


**Reposta questão 1**

Primeiramente vamos definir a função de base radial Multiquadrática inversa

In [139]:
def multiquadratica_inversa(x, c, s):
    return 1 / np.sqrt((x-c)**2 + s**2)

A seguir os ajustes no código para a rede rodar em cima da base Íris. Basicamente adicionamos um neurônio na camada de saída para cada classe. Um vez que a base Íris possui três classes, três neurônios foram adicionados na camada de saída

In [141]:
class RBFNet(object):
    """Implementation of a Radial Basis Function Network"""

    def __init__(self, k=3, attnumber=4, lr=0.01, epochs=100, rbf=rbfGaussiana, 
                 computeStds=computeEqualStds, n_classes=1):
        self.k = k  # grupos ou numero de neuronios na camada escondida
        self.lr = lr  # taxa de aprendizagem
        self.epochs = epochs  # número de iterações
        self.rbf = rbf  # função de base radial
        self.computeStds = computeStds  # função largura do campo receptivo
        self.w = [np.random.randn(self.k, attnumber) for _ in range(n_classes)]
        self.b = np.random.randn(n_classes)

    def fit(self, X, y):
        self.stds = []
        # 1º parâmetro da rede RBF
        kmeans = KMeans(
            n_clusters=self.k, init='random',
            n_init=10, max_iter=300).fit(X)
        self.centers = kmeans.cluster_centers_

        # 2º parâmetro da rede RBF (largura do campo receptivo)
        self.stds = self.computeStds(self.centers, self.k)
        for epoch in range(self.epochs):
            loss_rate = 0
            for i in range(X.shape[0]):
                phi = np.array([self.rbf(X[i], c, s)
                                for c, s, in zip(self.centers, self.stds)])
                # calcula somatório do produto da saída da função de base radial e os pesos
                output = []
                for _w, _b in zip(self.w, self.b):
                    F = np.sum(phi.T.dot(_w))+_b
                    output.append(0 if F < 0 else 1)
                # função de perda
                loss = np.sum((y[i] - output) ** 2)
                loss_rate += loss
                # cálculo do erro
                errors = y[i] - output
                # 3º Parâmetro da rede (atualização dos pesos)
                for j in range(len(self.b)):
                    self.w[j] = self.w[j] + self.lr * errors[j] * phi
                    self.b[j] = self.b[j] + self.lr * errors[j]
            # print(loss_rate)

    # calcula saída da rede RBF com a rede treinada
    def predict(self, X):
        y_pred = []
        error = 0
        for i in range(X.shape[0]):
            phi = np.array([self.rbf(X[i], c, s)
                            for c, s, in zip(self.centers, self.stds)])
            output = []
            for _w, _b in zip(self.w, self.b):
                F = np.sum(phi.T.dot(_w))+_b
                output.append(0 if F < 0 else 1)
            y_pred.append(output)
        return np.array(y_pred)

Execução da rede RBF com a base Íris. Para avaliar os resultados, cada configuração foi executada 10 vezes. A média e o desvio padrão foram calculados para melhor avaliação dos resultados


In [149]:
from sklearn.preprocessing import OneHotEncoder

# chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()
# transformação dos rótulos do conjunto de treinamento e teste
enc = OneHotEncoder(handle_unknown='ignore')
training_labels_transformed = enc.fit_transform(
    training_labels.values.reshape(-1, 1)).toarray()
test_labels_transformed = enc.fit_transform(
    test_labels.values.reshape(-1, 1)).toarray()
# parâmetros do número de neuronios na camada escondida
n_neurons = [3, 5, 7, 9]
for n in n_neurons:
    outputs=[]
    print("Rede RBF com {} neurônios na camada escondida".format(n))
    for _ in range(10):
        # iniciando a RBF
        rbfnet = RBFNet(lr=1e-2, attnumber=4, k=n, rbf=multiquadratica_inversa,
                        computeStds=computeEqualStds, n_classes=3)
        # treinamento da RBF
        rbfnet.fit(training_inputs.values, training_labels_transformed)
        # predição
        y_pred = rbfnet.predict(test_inputs.values)
        # cálculo da taxa de acerto e erro
        rate = [0 if False in (_p == _y) else 1 for _p, _y in zip(
            test_labels_transformed, y_pred)]
        outputs.append(np.sum(rate)/len(test_labels_transformed))
        print('\tTaxa de acerto: ', np.sum(
            rate)/len(test_labels_transformed))
        print('\tTaxa de erro: ', 1 -
            (np.sum(rate)/len(test_labels_transformed)))
    print('\tMédia taxa de acerto: ', np.mean(outputs))
    print('\tMédia taxa de erro: ', 1-np.mean(outputs))
    print('\tDesvio Padrão: ', np.std(outputs))


Rede RBF com 3 neurônios na camada escondida
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.7666666666666667
	Taxa de erro:  0.23333333333333328
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.7666666666666667
	Taxa de erro:  0.23333333333333328
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Média taxa de acerto:  0.8066666666666666
	Média taxa de erro:  0.19333333333333336
	Desvio Padrão:  0.024944382578492935
Rede RBF com 5 neurônios na camada escondida
	Taxa de acerto:  0.7333333333333333
	Taxa de erro:  0.2666666666666667
	Taxa de acerto:  0.7333333333333333
	Taxa de


Os resutados variaram pouco e não houve uma diferença significativa da acurácia entre as configurações, entretanto há uma tendência de uma menor taxa de erro para a configuração com 9 neurônios na camada escondida dentre as quatro opções. A seguir a rede é avaliada com a quantidade maior de neurônios na camada escondida

In [150]:
from sklearn.preprocessing import OneHotEncoder

# chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()
# transformação dos rótulos do conjunto de treinamento e teste
enc = OneHotEncoder(handle_unknown='ignore')
training_labels_transformed = enc.fit_transform(
    training_labels.values.reshape(-1, 1)).toarray()
test_labels_transformed = enc.fit_transform(
    test_labels.values.reshape(-1, 1)).toarray()
# parâmetros do número de neuronios na camada escondida
n_neurons = [20, 30, 50]
for n in n_neurons:
    outputs=[]
    print("Rede RBF com {} neurônios na camada escondida".format(n))
    for _ in range(10):
        # iniciando a RBF
        rbfnet = RBFNet(lr=1e-2, attnumber=4, k=n, rbf=multiquadratica_inversa,
                        computeStds=computeEqualStds, n_classes=3)
        # treinamento da RBF
        rbfnet.fit(training_inputs.values, training_labels_transformed)
        # predição
        y_pred = rbfnet.predict(test_inputs.values)
        # cálculo da taxa de acerto e erro
        rate = [0 if False in (_p == _y) else 1 for _p, _y in zip(
            test_labels_transformed, y_pred)]
        outputs.append(np.sum(rate)/len(test_labels_transformed))
        print('\tTaxa de acerto: ', np.sum(
            rate)/len(test_labels_transformed))
        print('\tTaxa de erro: ', 1 -
            (np.sum(rate)/len(test_labels_transformed)))
    print('\tMédia taxa de acerto: ', np.mean(outputs))
    print('\tMédia taxa de erro: ', 1-np.mean(outputs))
    print('\tDesvio Padrão: ', np.std(outputs))

Rede RBF com 20 neurônios na camada escondida
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.9
	Taxa de erro:  0.09999999999999998
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.9
	Taxa de erro:  0.09999999999999998
	Taxa de acerto:  0.8666666666666667
	Taxa de erro:  0.1333333333333333
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.8666666666666667
	Taxa de erro:  0.1333333333333333
	Taxa de acerto:  0.9
	Taxa de erro:  0.09999999999999998
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Média taxa de acerto:  0.8566666666666668
	Média taxa de erro:  0.1433333333333332
	Desvio Padrão:  0.0334995854037363
Rede RBF com 30 neurônios na camada escondida
	Taxa de acerto:  0.9
	Taxa de erro:  0.09999999999999998
	Taxa de acerto:  0.9
	Taxa de erro:  0.09999999999999998
	Tax

Com a quantidade de neurônios maior é possivel visualizar uma melhoria na acurácia. Isso se explica pela maior robustez do classificador, por meio da combinação de mais funções de base radial, no ajuste de regiões de segregação das classes no espaço de características. Outros parâmetros, como por exemplo, a quantidade de épocas, taxa de aprendizagem, parâmetros do K-means e entre outros também precisariam ser avaliados para melhor ajuste da rede

**Reposta questão 2**

A função abaixo mostra uma opção para calcular a largura do campo receptivo da função de base radial para cada neurônio


In [153]:
def compute_differents_stds(centers, k):
    dist = [np.sqrt(np.sum((c1 - c2) ** 2))
            for c1 in centers for c2 in centers if False in (c1 == c2)]
    dMin = np.min(dist)
    stds = np.repeat(dMin / 2, k)
    return stds

Calculando a taxa de erro usando a configuração com 9 neurônios na camada oculta e com a largura individual do campo receptivo dos neurônios


In [154]:
from sklearn.preprocessing import OneHotEncoder

# chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()
# transformação dos rótulos do conjunto de treinamento e teste
enc = OneHotEncoder(handle_unknown='ignore')
training_labels_transformed = enc.fit_transform(
    training_labels.values.reshape(-1, 1)).toarray()
test_labels_transformed = enc.fit_transform(
    test_labels.values.reshape(-1, 1)).toarray()
best=9
print("Rede RBF com {} neurônios na camada escondida".format(best))
# iniciando a RBF
rbfnet = RBFNet(lr=1e-2, attnumber=4, k=best, rbf=multiquadratica_inversa,
                computeStds=compute_differents_stds, n_classes=3)
# treinamento da RBF
rbfnet.fit(training_inputs.values, training_labels_transformed)
# predição
y_pred = rbfnet.predict(test_inputs.values)
# cálculo da taxa de acerto e erro
rate = [0 if False in (_p == _y) else 1 for _p, _y in zip(
    test_labels_transformed, y_pred)]
print('\tTaxa de acerto: ', np.sum(
    rate)/len(test_labels_transformed))
print('\tTaxa de erro: ', 1 -
    (np.sum(rate)/len(test_labels_transformed)))


Rede RBF com 9 neurônios na camada escondida
	Taxa de acerto:  0.9
	Taxa de erro:  0.09999999999999998


**Reposta questão 3**


In [155]:
from sklearn.preprocessing import OneHotEncoder

# chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()
# transformação dos rótulos do conjunto de treinamento e teste
enc = OneHotEncoder(handle_unknown='ignore')
training_labels_transformed = enc.fit_transform(
    training_labels.values.reshape(-1, 1)).toarray()
test_labels_transformed = enc.fit_transform(
    test_labels.values.reshape(-1, 1)).toarray()

_functions = [rbfGaussiana, multiquadratica_inversa]
_stds = [computeEqualStds, compute_differents_stds]
best=9
for _function in _functions:
    for _std in _stds:
        print("Função de base radial: ", _function.__name__)
        print("Função da largura do campo receptivo: ", _std.__name__)
        outputs=[]
        for _ in range(10):
            # iniciando a RBF
            rbfnet = RBFNet(lr=1e-2, attnumber=4, k=best, rbf=_function,
                            computeStds=_std, n_classes=3)
            # treinamento da RBF
            rbfnet.fit(training_inputs.values, training_labels_transformed)
            # predição
            y_pred = rbfnet.predict(test_inputs.values)
            # cálculo da taxa de acerto e erro
            rate = [0 if False in (_p == _y) else 1 for _p, _y in zip(
                test_labels_transformed, y_pred)]
            outputs.append(np.sum(rate)/len(test_labels_transformed))
            print('\tTaxa de acerto: ', np.sum(
                rate)/len(test_labels_transformed))
            print('\tTaxa de erro: ', 1 -
                (np.sum(rate)/len(test_labels_transformed)))
        print('\tMédia taxa de acerto: ', np.mean(outputs))
        print('\tMédia taxa de erro: ', 1-np.mean(outputs))
        print('\tDesvio Padrão: ', np.std(outputs))

Função de base radial:  rbfGaussiana
Função da largura do campo receptivo:  computeEqualStds
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Taxa de acerto:  0.7666666666666667
	Taxa de erro:  0.23333333333333328
	Taxa de acerto:  0.8666666666666667
	Taxa de erro:  0.1333333333333333
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Taxa de acerto:  0.8666666666666667
	Taxa de erro:  0.1333333333333333
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Taxa de acerto:  0.8333333333333334
	Taxa de erro:  0.16666666666666663
	Taxa de acerto:  0.8
	Taxa de erro:  0.19999999999999996
	Taxa de acerto:  0.9
	Taxa de erro:  0.09999999999999998
	Média taxa de acerto:  0.8266666666666665
	Média taxa de erro:  0.17333333333333345
	Desvio Padrão:  0.038873012632301994
Função de base radial:  rbfGaussiana
Função da largura do campo receptivo:  compute_differents_stds
	Taxa de acerto:  0.76666666666

Apesar dos resultados variarem pouco, é possível observar que para essa base de dados, alterando apenas a função da largura do campo receptivo e mantendo os outros parâmetros da rede, há uma tendência das combinações que envolvem o cálculo individual dos campos receptivos obterem uma melhor acurácia. Nesse contexto, a melhor configuração foi realizada pela combinação da função de base radial multiquadrática inversa junto com a função da largura do campo receptivo individual.